In [ ]:
# import cProfile
# import collections
import datetime

import os
# import sys
import time

# import numpy as np
import pandas as pd

import pickle

# from datetime import datetime
# from dotenv import load_dotenv

import gc

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})
format = '%d/%m/%Y %H:%M'

# 2. Concatenate all the playlists and create dictionaries for the main features data

## 2.1 Raw playlists

In [ ]:
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_full.pkl'
txt = 'All playlists concatenated - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_full_by_user = pickle.load(f)
else:
    print('Saved: ' + txt)
    playlists_full_by_user = []
    for i in range(0,100,1):
        data_path_aux = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(i) + '.pkl'
        if os.path.exists(data_path_aux):
            print(data_path_aux)
            with open(data_path_aux, 'rb') as f:
                playlist = pickle.load(f)
            playlists_full_by_user.extend(playlist)

    with open(data_path, 'wb') as f:
        pickle.dump(playlists_full_by_user, f)

print(len(playlists_full_by_user))

## 2.2 Create dictionary for all Playlists

In [ ]:
def get_artists(raw_artist):
    artists = []
    for a in raw_artist:
        artists.append({'id': a['id'],
                'name': a['name']})
    return(artists);

# a = get_artists(playlists_full_by_user[6]['tracks']['items'][2]['track']['album']['artists'])
# a

In [ ]:
def get_track_details(raw_track):
    if raw_track['track'] is not None:
        artists = get_artists(raw_track['track']['album']['artists'])
        track = raw_track['track']
        album = {
                'id': track['album']['id'],
                'name': track['album']['name'],
                'release_date': track['album']['release_date']
                }
        data = {
                    'id': track['id'],
                    'name': track['name'],
                    'added_at': raw_track['added_at'],
                    'popularity': track['popularity'],
                    'artists': artists,
                    'album': album
                    }
        return(data);

# a = get_track_details(playlists_full_by_user[6]['tracks']['items'][82])
# a['album']
# a = get_track_details(playlists_full_by_user[6]['tracks']['items'][83])
# playlists_full_by_user[6]['tracks']['items'][83]

In [ ]:
def get_playlist_detail(playlist):
    tracks = []
    for i, item in enumerate(playlist['tracks']['items']):
        if not(item['track'] is None or item['track']['id'] is None):
            track = get_track_details(item)
            tracks.append(track)
    data = {
                'user_id': playlist['owner']['id'],
                'playlist_id': playlist['id'],
                'total_tracks': len(tracks),
                'tracks': tracks
        }

    return(data);
# t = get_playlist_detail(playlists_full_by_user[4])
# t['tracks'][0]

In [ ]:
gc.collect();
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_3_playlists_dict.pkl'
txt = 'All playlists as dictionary - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_dict = pickle.load(f)
else:
    # Concatenate all playlists as dictionary
    n = len(playlists_full_by_user)

    playlists_dict = {}
    for i, playlist in enumerate(playlists_full_by_user):
        if i % 100 == 0:
            print("%s - %d/%d - LEN: %d" % (datetime.datetime.now().strftime(format), i, n, len(playlists_dict)))

        play_dict = get_playlist_detail(playlist)
        playlists_dict.update({playlist['id']: play_dict})

    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(playlists_dict, f)

print(len(playlists_dict))  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED: DUPLICATED PLAYLISTS

In [ ]:
cnt = []
n = len(playlists_full_by_user)
for i, playlist in enumerate(playlists_full_by_user):
#     if i % 100 == 0:
#         print("%s - %d/%d - LEN: %d" % (datetime.datetime.now().strftime(format), i, n, len(playlists_full_by_user)))

    play_dict = get_playlist_detail(playlist)
    cnt.append(playlist['id'])
    
df = pd.DataFrame (cnt, columns = ['id'])
count = pd.DataFrame(df.groupby('id').size(),columns=['cnt'])
a = count.groupby('cnt').size()
print(a)
a.sum()

## 2.3 Create dictionary for all Tracks

In [ ]:
gc.collect();
# All tracks
data_path = os.getcwd() + '\\data\\external\\raw_4_tracks_dict.pkl'
txt = 'All tracks as a dictionary - ' + data_path

if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        tracks_dict = pickle.load(f)
else:
    
    # Extract the user and playlist IDs
    tracks_dict = {}
    for i, pid in enumerate(playlists_dict):
        pl = playlists_dict.get(pid)
        for j, tr in enumerate(pl['tracks']):
            tid = tr['id']
    #         print(str(pid)  + ':' + str(i) + ' - ' + str(tid) + ':' +str(j))
            if tid not in tracks_dict.keys():
                tr['in_playlist'] = [pid]
                tracks_dict.update({tid: tr})
            else:
                curr_track = tracks_dict.get(tid)
                curr_in_playlist = curr_track['in_playlist'].copy()
                curr_in_playlist.append(pid)
                tr['in_playlist'] = curr_in_playlist
                tracks_dict.update({tid: tr})
    #             .append(tid)
    #             tr['in_playlist'].append(pid)

    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(tracks_dict, f)

print(len(tracks_dict))  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED

In [ ]:
# cnt = []
# n = len(playlists_full_by_user)
# for i, pid in enumerate(playlists_dict):
#         pl = playlists_dict.get(pid)
#         for j, tr in enumerate(pl['tracks']):
#             tid = tr['id']
#             cnt.append(tid)
    
# df = pd.DataFrame (cnt, columns = ['id'])
# count = pd.DataFrame(df.groupby('id').size(),columns=['cnt'])
# a = count.groupby('cnt').size()
# a.sum()

## 2.4 Create dictionary for all Artists

In [ ]:
gc.collect();
# All artists
data_path = os.getcwd() + '\\data\\external\\raw_5_artists_dict.pkl'
txt = 'All artists as a dictionary - ' + data_path

if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        artists_dict = pickle.load(f)
else:
    # Extract the user and playlist IDs
    artists_dict = {}
    stop = 1
    for i, pid in enumerate(playlists_dict):
        pl = playlists_dict.get(pid)
        for j, tr in enumerate(pl['tracks']):
            for k, art in enumerate(tr['artists']):
                aid = art['id']
                if aid not in artists_dict.keys():
                    art['in_playlist'] = [pid]
                    artists_dict.update({aid: art})
                else:
                    curr_artist = artists_dict.get(aid)
#                     if stop ==1:
#                         print(pid, tr, aid, curr_artist)
#                     stop = 2
                    curr_in_playlist = curr_artist['in_playlist'].copy()
                    curr_in_playlist.append(pid)
                    art['in_playlist'] = curr_in_playlist
                    artists_dict.update({aid: art})
                    
    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(artists_dict, f)

print(len(artists_dict))
# artists_dict

## 3.5 Create dictionary for all Albums

In [ ]:
gc.collect();
# All albums
data_path = os.getcwd() + '\\data\\external\\raw_6_albums_dict.pkl'
txt = 'All albums as a dictionary - ' + data_path

if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        albums_dict = pickle.load(f)
else:
    
    # Extract the user and playlist IDs
    albums_dict = {}
    for i, pid in enumerate(playlists_dict):
        pl = playlists_dict.get(pid)
        for j, tr in enumerate(pl['tracks']):
            alb = tr['album']
            aid = alb['id']
            if aid not in albums_dict.keys():
                alb['in_playlist'] = [aid]
                albums_dict.update({aid: alb})
            else:
                curr_album = albums_dict.get(aid)
                curr_in_playlist = curr_album['in_playlist'].copy()
                curr_in_playlist.append(pid)
                alb['in_playlist'] = curr_in_playlist
                albums_dict.update({aid: alb})

    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(albums_dict, f)

print(len(albums_dict))
# albums_dict